In [46]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [2]:
training_data = torchvision.datasets.MNIST('/files/', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.MNIST('/files/', train=False, download=True, transform=torchvision.transforms.ToTensor())

In [3]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
log_interval = 100

In [5]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size_test, shuffle=True)

In [6]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
image_dimensions = example_data[0][0].size()

In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.avg1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.avg2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.avg1(F.sigmoid(self.conv1(x)))
        x = self.avg2(F.sigmoid(self.conv2(x)))
        x = F.sigmoid(self.fc1(self.flatten(x)))
        x = F.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return x


In [56]:
net = Net()
net.cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [42]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [43]:
def train(epoch):
    net.train()
    for batch_idx, (x, y) in enumerate(train_loader):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        y_hat = net(x)
        l = loss(y_hat, y)
        l.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), l.item()))
            train_losses.append(l.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [44]:
def test():
  net.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for x, y in test_loader:
      x = x.cuda()
      y = y.cuda()
      y_hat = net(x)
      test_loss +=  loss(y_hat, y).item()
      pred = y_hat.data.max(1, keepdim=True)[1]
      correct += pred.eq(y.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [57]:
for epoch in range(0, n_epochs):
  train(epoch)
  test()

C:\Users\Usuario\.conda\envs\vg\lib\site-packages\torch\nn\functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


RuntimeError: mat1 dim 1 must match mat2 dim 0

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = nn.Sequential(nn.Flatten(), nn.Linear(input_dimension, input_dimension), nn.ReLU(), nn.Linear(input_dimension, ouput_dimension), nn.ReLU())
    def forward(self, x):
        return self.net(x)

In [21]:
net = Net()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
for epoch in range(0, 5):
  train(epoch)
  test()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.307909
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.103603
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.834737
Train Epoch: 0 [19200/60000 (32%)]	Loss: 1.642663
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.327896
Train Epoch: 0 [32000/60000 (53%)]	Loss: 1.223220
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.111410
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.908534
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.911270
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.939589

Test set: Avg. loss: 0.0008, Accuracy: 7727/10000 (77%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.864560
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.694731
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.841547
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.627143
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.674995
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.440828
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.441563
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.535328
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.496050
Trai